# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json, re
import gmaps
import os

# Import API keys
from config import (ow_key, gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [419]:
file = "../WeatherPy/output_data/cities_weather_data.csv"
weather_data_df = pd.read_csv(file, encoding="utf-8")

weather_data_df.drop('Degrees From Equator', inplace=True, axis=1)
weather_data_df.head()


,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Shellbrook,CA,53.2168,-106.4011,7.0,49.0,75.0,7.72,26/04/21
1,Vila Velha,BR,-20.3297,-40.2925,24.0,94.0,40.0,2.41,26/04/21
2,Nizhniy Kuranakh,RU,58.8353,125.4869,1.0,97.0,100.0,5.55,26/04/21
3,Ahmadpur East,PK,29.1431,71.2598,37.0,6.0,0.0,3.77,26/04/21
4,George Town,KY,5.4112,100.3354,28.0,78.0,20.0,1.54,26/04/21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [420]:
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_df[["Latitude", "Longitude"]]

humidity = weather_data_df["Humidity"]

In [421]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(0.0, 30.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weather_data_df["Humidity"].max(),
                                 point_radius = 3)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [428]:
ideal_vacay_spots_df = weather_data_df.loc[(weather_data_df['Temperature'] >= 21) &  
                                        (weather_data_df['Temperature'] <= 27) &
                                        (weather_data_df['Wind Speed'] <= 5) & 
                                        (weather_data_df['Cloudiness'] <= 10)
                                        ].reset_index(drop=True)

ideal_vacay_spots_df[['Hotel Name','Country']] = ''
ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Port Lincoln,AU,-34.7333,135.8667,21.0,53.0,0.0,2.26,26/04/21,,
1,Hilo,US,19.7297,-155.0900,24.0,69.0,1.0,2.57,26/04/21,,
2,Buraydah,SA,26.3260,43.9750,24.0,53.0,0.0,2.06,26/04/21,,
3,Plant City,US,28.0186,-82.1129,22.0,73.0,1.0,4.12,26/04/21,,
4,Gibara,CU,21.1097,-76.1317,24.0,88.0,0.0,2.57,26/04/21,,
5,Pishin,PK,30.5803,66.9961,25.0,13.0,0.0,0.14,26/04/21,,
6,Shache,CN,38.4167,77.2406,21.0,14.0,0.0,3.23,26/04/21,,
7,Springdale,CA,36.1867,-94.1288,21.0,52.0,1.0,4.63,26/04/21,,
8,Dali,CN,25.7000,100.1833,26.0,21.0,2.0,2.30,26/04/21,,
9,Monte Aprazivel,BR,-20.7725,-49.7142,23.0,56.0,0.0,1.54,26/04/21,,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [433]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_vacay_spots_df.iterrows():

    params = {
    "location": f"{row[2]},{row[3]}",
    "radius": 5000,
    "type": "lodging",
    "keyword" : "hotel",
    "language" : "en",
    "key": gkey
    }
    
    try:
        first_hotel = requests.get(base_url, params=params).json()['results'][0]
        ideal_vacay_spots_df.at[index, 'Hotel Name'] = first_hotel['name']
        country, = re.findall('\s.*,\s(.*?)$', first_hotel['plus_code']['compound_code'])
        ideal_vacay_spots_df.at[index, 'Country'] = country
    except (KeyError, ValueError):
                ideal_vacay_spots_df.at[index, 'Country'] = row['Country Code']

ideal_vacay_spots_df

,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Country
0,Port Lincoln,AU,-34.7333,135.8667,21.0,53.0,0.0,2.26,26/04/21,Port Lincoln Hotel,Australia
1,Hilo,US,19.7297,-155.0900,24.0,69.0,1.0,2.57,26/04/21,Hilo Hawaiian Hotel,United States
2,Buraydah,SA,26.3260,43.9750,24.0,53.0,0.0,2.06,26/04/21,Mövenpick Hotel Qassim,SA
3,Plant City,US,28.0186,-82.1129,22.0,73.0,1.0,4.12,26/04/21,Hampton Inn Plant City,United States
4,Gibara,CU,21.1097,-76.1317,24.0,88.0,0.0,2.57,26/04/21,Hotel E Plaza Colón,Cuba
5,Pishin,PK,30.5803,66.9961,25.0,13.0,0.0,0.14,26/04/21,Haji Abdul Samad Hotel,Pakistan
6,Shache,CN,38.4167,77.2406,21.0,14.0,0.0,3.23,26/04/21,Xinsheng Hotel,China
7,Springdale,CA,36.1867,-94.1288,21.0,52.0,1.0,4.63,26/04/21,Holiday Inn Springdale/Fayetteville Area,United States
8,Dali,CN,25.7000,100.1833,26.0,21.0,2.0,2.30,26/04/21,Sunyata Hotel,China
9,Monte Aprazivel,BR,-20.7725,-49.7142,23.0,56.0,0.0,1.54,26/04/21,Torres Palace Hotel,Brazil


In [434]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in ideal_vacay_spots_df.iterrows()]
locations = ideal_vacay_spots_df[["Latitude", "Longitude"]]

In [435]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers

markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))